In [ ]:
## impute donation type for deferrals for both US and SA data before preprocessing


import pyreadstat
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#SANBS

df=pd.read_csv("../1_data/private/SANBSdata.csv")
len(df)

In [ ]:
#categorise donation types

df.loc[(df['donation_product'] == 'WB')|
           (df['donation_product'] == 'WBSB')| (df['donation_product'] == 'WBMED')| (df['donation_product'] == 'WBDISC')
           |( df['donation_product'] == 'WBOB') |(df['donation_product'] == 'WBDP') | (df['donation_product'] == 'WBD') ,
           'donation_product']='WB'

df.loc[(df['donation_product'] == 'RBCAPH')|
           (df['donation_product'] == 'HPC.APHERESIS')| (df['donation_product'] == 'APHRET_PLASPLCA')| (df['donation_product'] == 'APHRET_PLCA'),'donation_product']='APH'

df.loc[(df['donation_product'] == 'SPCCPSTOP')|
           (df['donation_product'] == 'CCPD')| (df['donation_product'] == 'SPCCPRETURN')| (df['donation_product'] == 'SPCCP'),
           'donation_product']='CCP'

In [ ]:

# Compute donation product for deferred donors - using the following criteria
 # 1) if donor only has WB then deferred donation is WB
 # 2) if next donation procedure is WB then deferred donation is WB/
 # 3) if previous donation procedure is WB then deferred donation is WB/
 # 4) if only donated one time and got deferred - WB
 
#prev successful 
#next successful
df.sort_values(by=['DonorID', 'don_Sequence', 'Visit_Date'], axis = 0, inplace=True)

df["NEXT_DON"] = np.where(df['DonorID'] == df['DonorID'].shift(-1), df['donation_product'].shift(-1), 10)
df["PREV_DON"] = np.where(df['DonorID'] == df['DonorID'].shift(1), df['donation_product'].shift(1), 10)


df['def_prod_imputed'] = np.where((df['donation_product'] == 'DEF PROD') & (df.PREV_DON=='WB'), 1, 0)
df['def_prod_imputed'] = np.where((df['donation_product'] == 'DEF PROD') & (df.NEXT_DON=='WB'), 1, df.def_prod_imputed)
df['def_prod_imputed'] = np.where((df['donation_product'] == 'DEF PROD') & ((df.NEXT_DON==10) & (df.PREV_DON==10)), 1, df.def_prod_imputed)
df['donation_product']=np.where(df.def_prod_imputed == 1, 'WB', df.donation_product)

# backward fill remaining NOT DRAWNs- most proximal next visit
df['donation_product']=np.where((df['donation_product'] == 'DEF PROD'), np.NaN, (df['donation_product']))

df['donation_product'] = df.groupby('DonorID')['donation_product'].bfill()


#keep only data after 2017
mask = (df['Visit_yr'] >= 2017) 
df = df.loc[mask]


In [ ]:
#save preprocessed file
df.to_csv(("../1_data/private/SANBSdata2.csv"), index=False)

In [ ]:
#######################################################

In [ ]:
#Vitalant
don=pd.read_csv("../1_data/private/donation_donor.csv") #donations

In [ ]:
#categorise donation types

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Whole Blood'), 'don_proc_cat']='WB'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'RBCP') |
          (don['PHLEBOTOMY_GROUP'] == 'Platelet/RBC/Plasma') | (don['PHLEBOTOMY_GROUP'] == 'Platelet/RBC'), 'don_proc_cat']='RBC'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Platelet') | (don['PHLEBOTOMY_GROUP'] == 'Platelet/Plasma') |
          (don['PHLEBOTOMY_GROUP'] == 'Plasma') | (don['PHLEBOTOMY_GROUP'] == 'COVID-19 Plasma') | (don['PHLEBOTOMY_GROUP'] == 'Source Plasma'), 'don_proc_cat']='Platelet/Plasma'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'COVID-19 Plasma'), 'don_proc_cat']='CCP'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Sample Only') | (don['PHLEBOTOMY_GROUP'] == 'Research/Sample Only'), 'don_proc_cat']='Sample'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Not Drawn') | (don['PHLEBOTOMY_GROUP'] == 'UNKNOWN'), 'don_proc_cat']='Not Drawn/Unknown'

don.loc[(don['PHLEBOTOMY_GROUP'] == '2RBC') , 'don_proc_cat']='2RBC'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Tissue Collections') | (don['PHLEBOTOMY_GROUP'] == 'Granulocyte'), 'don_proc_cat']='Tissue/Granulocyte'

don.don_proc_cat.value_counts()


In [ ]:
#store next and orev donation type
don.sort_values(by=['DONOR_KEY', 'DON_DATE_KEY', 'DIN'], axis = 0, inplace=True)
don["NEXT_DON"] = np.where(don['DONOR_KEY'] == don['DONOR_KEY'].shift(-1), don['don_proc_cat'].shift(-1), 10)
don["PREV_DON"] = np.where(don['DONOR_KEY'] == don['DONOR_KEY'].shift(1), don['don_proc_cat'].shift(1), 10)

#IMPUTE DON_PROC
# if immediate next donation procedure is WB/RBC then deferred donation is WB/RBC
 #if immediate previous donation procedure is WB/RBC then deferred donation is WB/RBC
# if only donated one time and got deferred - WB/RBC


don['def_prod_imputed'] = np.where((don['don_proc_cat'] == 'Not Drawn/Unknown') & (don.PREV_DON=='WB'), 1, 0)
don['def_prod_imputed'] = np.where((don['don_proc_cat'] == 'Not Drawn/Unknown') & (don.NEXT_DON=='WB'), 1, don.def_prod_imputed)
don['def_prod_imputed'] = np.where((don['don_proc_cat'] == 'Not Drawn/Unknown') & ((don.NEXT_DON==10) & (don.PREV_DON==10)), 1, don.def_prod_imputed)

don['don_proc_cat']=np.where(don.def_prod_imputed == 1, 'WB', don.don_proc_cat)

# backward fill remaining NOT DRAWNs- most proximal next visit

don['don_proc_cat']=np.where((don['don_proc_cat'] == 'Not Drawn/Unknown'), np.NaN, don.don_proc_cat)

don['don_proc_cat'] = don.groupby('DONOR_KEY')['don_proc_cat'].bfill()
don.don_proc_cat.value_counts()

#don=don2#uncomment and run to reset
don2=don #duplicate before changing
don.drop(don[(don['DONOR_NUMBER'] == 'UNKNOWN')].index, inplace=True)
don.drop(don[(don['DONOR_KEY'] == 0.0)].index, inplace=True)
don.drop(don[don.DON_DATE_KEY.isna()].index, inplace=True)

don['FIN']=don.DONATION_NUMBER.str.slice(start=0, stop=5)
don.drop(don[(don['FIN'] == 'W0358') |
             (don['FIN'] == 'W0364') |
             (don['FIN'] == 'W0433') |
             (don['FIN'] == 'W2864') |
             (don['FIN'] == 'W2865') |
             (don['FIN'] == 'W2868') |
             (don['FIN'] == 'W2869') |
             (don['FIN'] == 'W2870') |
             (don['FIN'] == 'W2873') |
             (don['FIN'] == 'W2874') |
             (don['FIN'] == 'W2875') |
             (don['FIN'] == 'W2876') |
             (don['FIN'] == 'W2877') |
             (don['FIN'] == 'W2878') |
             (don['FIN'] == 'W2879') |
             (don['FIN'] == 'W2880') |
             (don['FIN'] == 'W2881') |
             (don['FIN'] == 'W2882') |
             (don['FIN'] == 'W2883') |
             (don['FIN'] == 'W2884') |
             (don['FIN'] == 'W2885') |
             (don['FIN'] == 'W2886') ].index, inplace=True)

len(don)

In [ ]:
#save preprocessed file
don.to_csv(("../1_data/private/donation_donor2.csv"), index=False)